In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import joblib
from scipy.sparse import hstack

In [7]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
df = pd.read_csv(next(iter(uploaded)))


Saving workflows_dataset.csv to workflows_dataset.csv


In [12]:
# Vérifier les premières lignes du dataset
print(df.head())

                                        workflowName  steps  actions
0  Recruitment Workflow,"Identify the need, Job p...    NaN      NaN
1  IT Ticket Management Workflow,"Ticket creation...    NaN      NaN
2  New Product Launch Workflow,"Idea generation &...    NaN      NaN
3  Sales Process Workflow,"Lead generation, Initi...    NaN      NaN
4  Invoice Processing Workflow,"Invoice reception...    NaN      NaN


In [10]:
# Étape 2 : Prétraitement des données textuelles
# Initialiser le TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')

In [14]:
# Convertir les valeurs NaN ou float en chaînes de caractères
df['steps'] = df['steps'].astype(str)
df['actions'] = df['actions'].astype(str)

# Appliquer le vectorizer
X_steps = vectorizer.fit_transform(df['steps'])
X_actions = vectorizer.fit_transform(df['actions'])


In [16]:
# Fusionner les matrices des étapes et des actions
X = hstack([X_steps, X_actions])

In [17]:
# Vérifier la forme du tableau combiné
print(X.shape)

(10, 2)


In [18]:
# Étape 3 : Séparation du dataset en ensemble d'entraînement et ensemble de test
y = df['workflowName']

In [19]:
# Séparer les données en entraînement et test (80% entraînement, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Vérifier les tailles des ensembles
print(X_train.shape, X_test.shape)

(8, 2) (2, 2)


In [21]:
# Étape 4 : Entraînement du modèle
# Initialiser le modèle RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [22]:
# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [52]:
# Étape 5 : Évaluation du modèle
# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

In [37]:
# Étape 6 : Hyperparameter tuning (optionnel)
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Change cv to a value less than the minimum number of samples in any class
# For example, if the smallest class has 3 samples, you can set cv to 3 or less.
# Using cv=y_train.nunique() if number of classes is less than number of samples in the smallest class
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)


In [39]:
# Afficher les meilleurs paramètres
print(f"Best Parameters: {grid_search}")

Best Parameters: GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             verbose=1)


In [40]:
# Étape 7 : Sauvegarde du modèle
joblib.dump(model, 'workflow_model.pkl')

['workflow_model.pkl']

In [55]:
# Étape 8 : Prédictions sur de nouvelles données
# Charger le modèle sauvegardé
model = joblib.load('workflow_model.pkl')

In [123]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# Exemple de données d'entraînement
training_data = [
    "Identify the need",
    "Job posting",
    "Perform assessments",
    "Interviews & assessments"
]

# Créer le vectorizer
vectorizer = CountVectorizer()

# Adapter le vectorizer sur les données d'entraînement
vectorizer.fit(training_data)

# Nouvelles données à vectoriser
new_data = ["Idea generation, Product design, Market research"]

# Transformer les nouvelles données en utilisant le vectorizer entraîné
X_new = vectorizer.transform(new_data)

# Convertir la matrice en un tableau dense pour visualiser les résultats
X_new_dense = X_new.toarray()

# Créer et entraîner le modèle (RandomForestClassifier pour exemple)
y_train = ['Recruitment Workflow', 'Project Management Workflow', 'HR Review Workflow', 'Event Planning Workflow']  # Remplacer par vos propres labels
model = RandomForestClassifier()
X_train = vectorizer.transform(training_data)  # Transformation des données d'entraînement
model.fit(X_train, y_train)

# Prédire le workflow
prediction = model.predict(X_new)

# Préparer la sortie sous forme de dictionnaire
output = {
    "shape_of_transformed_data": X_new.shape,
    "transformed_data_dense_matrix": X_new_dense,
    "vocabulary": vectorizer.get_feature_names_out(),
    "predicted_workflow": prediction[0]
}

# Afficher les résultats de manière structurée
print("="*50)
print("                         Output Structure                         ")
print("="*50)

print("\n1. Shape of the Transformed New Data:")
print(f"   {output['shape_of_transformed_data']}\n")


print("2. Vocabulary:")
print("   " + ", ".join(output['vocabulary']) + "\n")

print("3. Predicted Workflow:")
print(f"   {output['predicted_workflow']}\n")

print("="*50)


                         Output Structure                         

1. Shape of the Transformed New Data:
   (1, 8)

2. Vocabulary:
   assessments, identify, interviews, job, need, perform, posting, the

3. Predicted Workflow:
   HR Review Workflow

